In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
import requests
from io import StringIO

In [ ]:
# Set up data folder
DATA = Path('/content/drive/My Drive/data')

## Pathway mutation analysis

Use `KEGG REST API` to obtain all KEGG pathways and the genes.

In [ ]:
# Get pathway_id, gene_id table
KEGG_PATHWAY_LINK_BASE = "http://rest.kegg.jp/link/pathway/hsa"
pathway_link = requests.get(KEGG_PATHWAY_LINK_BASE)
kegg = pd.read_csv(StringIO(pathway_link.text), header=None, sep="\t")
kegg.columns = ["gene_id","pathway_id"]

# Get name of each pathway
KEGG_PATHWAY_LIST_BASE = "http://rest.kegg.jp/list/pathway/hsa"
pathway_list = requests.get(KEGG_PATHWAY_LIST_BASE)
kegg_name = pd.read_csv(StringIO(pathway_list.text), header=None,
                                    sep="\t")
kegg_name.columns = ["pathway_id","pathway_name"]

kegg = kegg.merge(kegg_name)
kegg.head()

,gene_id,pathway_id,pathway_name
0,hsa:10327,path:hsa00010,Glycolysis / Gluconeogenesis - Homo sapiens (h...
1,hsa:124,path:hsa00010,Glycolysis / Gluconeogenesis - Homo sapiens (h...
2,hsa:125,path:hsa00010,Glycolysis / Gluconeogenesis - Homo sapiens (h...
3,hsa:126,path:hsa00010,Glycolysis / Gluconeogenesis - Homo sapiens (h...
4,hsa:127,path:hsa00010,Glycolysis / Gluconeogenesis - Homo sapiens (h...


In [ ]:
maf_file = DATA /"TCGA.BRCA.mutect.995c0111-d90b-4140-bee7-3845436c3b42.DR-10.0.somatic.maf.gz"

pd.set_option('display.max_columns', None)
maf = pd.read_csv(DATA / maf_file, sep="\t", comment='#', low_memory=False)
maf.head()

,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,dbSNP_RS,dbSNP_Val_Status,Tumor_Sample_Barcode,Matched_Norm_Sample_Barcode,Match_Norm_Seq_Allele1,Match_Norm_Seq_Allele2,Tumor_Validation_Allele1,Tumor_Validation_Allele2,Match_Norm_Validation_Allele1,Match_Norm_Validation_Allele2,Verification_Status,Validation_Status,Mutation_Status,Sequencing_Phase,Sequence_Source,Validation_Method,Score,BAM_File,Sequencer,Tumor_Sample_UUID,Matched_Norm_Sample_UUID,HGVSc,HGVSp,HGVSp_Short,Transcript_ID,Exon_Number,t_depth,t_ref_count,t_alt_count,n_depth,n_ref_count,n_alt_count,all_effects,Allele,Gene,Feature,Feature_type,One_Consequence,Consequence,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,ALLELE_NUM,DISTANCE,TRANSCRIPT_STRAND,SYMBOL,SYMBOL_SOURCE,HGNC_ID,BIOTYPE,CANONICAL,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,RefSeq,SIFT,PolyPhen,EXON,INTRON,DOMAINS,GMAF,AFR_MAF,AMR_MAF,ASN_MAF,EAS_MAF,EUR_MAF,SAS_MAF,AA_MAF,EA_MAF,CLIN_SIG,SOMATIC,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,IMPACT,PICK,VARIANT_CLASS,TSL,HGVS_OFFSET,PHENO,MINIMISED,ExAC_AF,ExAC_AF_Adj,ExAC_AF_AFR,ExAC_AF_AMR,ExAC_AF_EAS,ExAC_AF_FIN,ExAC_AF_NFE,ExAC_AF_OTH,ExAC_AF_SAS,GENE_PHENO,FILTER,CONTEXT,src_vcf_id,tumor_bam_uuid,normal_bam_uuid,case_id,GDC_FILTER,COSMIC,MC3_Overlap,GDC_Validation_Status
0,USP24,23358,WUGSC,GRCh38,chr1,55159655,55159655,+,Missense_Mutation,SNP,T,T,C,rs150880897,by1000G;byCluster;byFrequency,TCGA-D8-A1XY-01A-11D-A14K-09,TCGA-D8-A1XY-10A-01D-A14K-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Somatic,NaN,NaN,NaN,NaN,NaN,Illumina HiSeq 2000,edb6d161-8f50-4c11-8246-487c4ea9a55d,8dea96d9-5017-4872-a84e-33bfd2f37b7a,c.1024A>G,p.Ile342Val,p.I342V,ENST00000294383,9/68,82,49,33,126,NaN,NaN,"USP24,missense_variant,p.I342V,ENST00000294383...",C,ENSG00000162402,ENST00000294383,Transcript,missense_variant,missense_variant,1024/10549,1024/7863,342/2620,I/V,Atc/Gtc,rs150880897,1,NaN,-1.0,USP24,HGNC,HGNC:12623,protein_coding,YES,CCDS44154.2,ENSP00000294383,Q9UPU5,NaN,UPI000059CFDE,NM_015306.2,tolerated(0.37),benign(0.003),9/68,NaN,NaN,0.0020,0.0076,0.0,NaN,0.0,0.0,0.0,0.0059,0.0005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MODERATE,1.0,SNV,5.0,NaN,NaN,1,0.000715,0.002251,0.009470,0.0,0.0,0.0,0.002752,0.003676,0.0,NaN,panel_of_normals,CTGGATTGTAG,d083d669-6646-463b-853e-c58da8d06439,4374e19d-c5e7-49cf-8707-05ae5aeb7369,aadee87c-6a68-4580-bd10-64ac273b1e3d,0130d616-885e-4a6c-9d03-2f17dd692a05,common_in_exac;gdc_pon,NaN,True,Unknown
1,ERICH3,127254,WUGSC,GRCh38,chr1,74571494,74571494,+,Missense_Mutation,SNP,C,C,T,NaN,NaN,TCGA-D8-A1XY-01A-11D-A14K-09,TCGA-D8-A1XY-10A-01D-A14K-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Somatic,NaN,NaN,NaN,NaN,NaN,Illumina HiSeq 2000,edb6d161-8f50-4c11-8246-487c4ea9a55d,8dea96d9-5017-4872-a84e-33bfd2f37b7a,c.4216G>A,p.Glu1406Lys,p.E1406K,ENST00000326665,14/15,86,42,44,64,NaN,NaN,"ERICH3,missense_variant,p.E1406K,ENST000003266...",T,ENSG00000178965,ENST00000326665,Transcript,missense_variant,missense_variant,4435/7159,4216/4593,1406/1530,E/K,Gag/Aag,NaN,1,NaN,-1.0,ERICH3,HGNC,HGNC:25346,protein_coding,YES,CCDS30755.1,ENSP00000322609,Q5RHP9,NaN,UPI0000237200,NM_001002912.4,deleterious(0.01),possibly_damaging(0.833),14/15,NaN,PROSITE_profiles:PS50313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MODERATE,1.0,SNV,5.0,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PASS,TTCCTCTACCA,d083d669-6646-463b-853e-c58da8d06439,4374e19d-c5e7-49cf-8707-05ae5aeb7369,aadee87c-6a68-4580-bd10-64ac273b1e3d,0130d616-885e-4a6c-9d03-2f17dd692a05,NaN,COSM1474194,True,Unknown
2,KIF26B,55083,WUGSC,GRCh38,chr1,245419680,245419680,+,Silent,SNP,G,G,T,NaN,NaN,TCGA-D8-A1XY-01A-11D-A14K-09,TCGA-D8-A1XY-10A-01D-A14K-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Somatic,NaN,NaN,NaN,NaN,NaN,Illumina HiSeq 2000,edb6d161-8f50-4c11-8246-487c4ea9a55d,8dea96d9-5017-4872-a84e-33bfd2f37b7a,c.1101G>T,p.=,p.S367S,ENST00000407071,

In [ ]:
# print('Filter:', maf.FILTER.unique())
# print('Mutation_Status:', maf.Mutation_Status.unique())
# print('Variant_Classification:', maf.Variant_Classification.unique())
# print('IMPACT:', maf.IMPACT.unique())

In [ ]:
# make sure the mutations pass the "filter" and they are not modifiers
mut = maf[(maf.FILTER == 'PASS') & (maf.IMPACT != 'MODIFIER')].copy()

print("{} out of {} mutations passed the filtering criteria, {:.2f}%.".format(
    len(mut), len(maf), 100 * len(mut) / len(maf)))

# mut.head()

95711 out of 120988 mutations passed the filtering criteria, 79.11%.


The KEGG pathway table's gene IDs has `hsa:` prefix to them.

In [ ]:
# add hsa to the id in maf table so it can match with kegg

mut_hsa = mut.copy()
mut_hsa.Entrez_Gene_Id = mut_hsa.apply(lambda x: 'hsa:'+ str(x.Entrez_Gene_Id), axis = 1)
len(mut_hsa)

95711

In [ ]:
mut_hsa.head()

,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,dbSNP_RS,dbSNP_Val_Status,Tumor_Sample_Barcode,Matched_Norm_Sample_Barcode,Match_Norm_Seq_Allele1,Match_Norm_Seq_Allele2,Tumor_Validation_Allele1,Tumor_Validation_Allele2,Match_Norm_Validation_Allele1,Match_Norm_Validation_Allele2,Verification_Status,Validation_Status,Mutation_Status,Sequencing_Phase,Sequence_Source,Validation_Method,Score,BAM_File,Sequencer,Tumor_Sample_UUID,Matched_Norm_Sample_UUID,HGVSc,HGVSp,HGVSp_Short,Transcript_ID,Exon_Number,t_depth,t_ref_count,t_alt_count,n_depth,n_ref_count,n_alt_count,all_effects,Allele,Gene,Feature,Feature_type,One_Consequence,Consequence,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,ALLELE_NUM,DISTANCE,TRANSCRIPT_STRAND,SYMBOL,SYMBOL_SOURCE,HGNC_ID,BIOTYPE,CANONICAL,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,RefSeq,SIFT,PolyPhen,EXON,INTRON,DOMAINS,GMAF,AFR_MAF,AMR_MAF,ASN_MAF,EAS_MAF,EUR_MAF,SAS_MAF,AA_MAF,EA_MAF,CLIN_SIG,SOMATIC,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,IMPACT,PICK,VARIANT_CLASS,TSL,HGVS_OFFSET,PHENO,MINIMISED,ExAC_AF,ExAC_AF_Adj,ExAC_AF_AFR,ExAC_AF_AMR,ExAC_AF_EAS,ExAC_AF_FIN,ExAC_AF_NFE,ExAC_AF_OTH,ExAC_AF_SAS,GENE_PHENO,FILTER,CONTEXT,src_vcf_id,tumor_bam_uuid,normal_bam_uuid,case_id,GDC_FILTER,COSMIC,MC3_Overlap,GDC_Validation_Status
1,ERICH3,hsa:127254,WUGSC,GRCh38,chr1,74571494,74571494,+,Missense_Mutation,SNP,C,C,T,NaN,NaN,TCGA-D8-A1XY-01A-11D-A14K-09,TCGA-D8-A1XY-10A-01D-A14K-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Somatic,NaN,NaN,NaN,NaN,NaN,Illumina HiSeq 2000,edb6d161-8f50-4c11-8246-487c4ea9a55d,8dea96d9-5017-4872-a84e-33bfd2f37b7a,c.4216G>A,p.Glu1406Lys,p.E1406K,ENST00000326665,14/15,86,42,44,64,NaN,NaN,"ERICH3,missense_variant,p.E1406K,ENST000003266...",T,ENSG00000178965,ENST00000326665,Transcript,missense_variant,missense_variant,4435/7159,4216/4593,1406/1530,E/K,Gag/Aag,NaN,1,NaN,-1.0,ERICH3,HGNC,HGNC:25346,protein_coding,YES,CCDS30755.1,ENSP00000322609,Q5RHP9,NaN,UPI0000237200,NM_001002912.4,deleterious(0.01),possibly_damaging(0.833),14/15,NaN,PROSITE_profiles:PS50313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MODERATE,1.0,SNV,5.0,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PASS,TTCCTCTACCA,d083d669-6646-463b-853e-c58da8d06439,4374e19d-c5e7-49cf-8707-05ae5aeb7369,aadee87c-6a68-4580-bd10-64ac273b1e3d,0130d616-885e-4a6c-9d03-2f17dd692a05,NaN,COSM1474194,True,Unknown
2,KIF26B,hsa:55083,WUGSC,GRCh38,chr1,245419680,245419680,+,Silent,SNP,G,G,T,NaN,NaN,TCGA-D8-A1XY-01A-11D-A14K-09,TCGA-D8-A1XY-10A-01D-A14K-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Somatic,NaN,NaN,NaN,NaN,NaN,Illumina HiSeq 2000,edb6d161-8f50-4c11-8246-487c4ea9a55d,8dea96d9-5017-4872-a84e-33bfd2f37b7a,c.1101G>T,p.=,p.S367S,ENST00000407071,4/15,85,29,56,59,NaN,NaN,"KIF26B,synonymous_variant,p.S367S,ENST00000407...",T,ENSG00000162849,ENST00000407071,Transcript,synonymous_variant,synonymous_variant,1541/7287,1101/6327,367/2108,S,tcG/tcT,NaN,1,NaN,1.0,KIF26B,HGNC,HGNC:25484,protein_coding,YES,CCDS44342.1,ENSP00000385545,Q2KJY2,NaN,UPI0000695D71,NM_018012.3,NaN,NaN,4/15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LOW,1.0,SNV,1.0,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PASS,GCCTCGCAGGG,d083d669-6646-463b-853e-c58da8d06439,4374e19d-c5e7-49cf-8707-05ae5aeb7369,aadee87c-6a68-4580-bd10-64ac273b1e3d,0130d616-885e-4a6c-9d03-2f17dd692a05,NaN,COSM1473725;COSM1473726,True,Unknown
3,USP34,hsa:9736,WUGSC,GRCh38,chr2,61189055,61189055,+,Silent,SNP,G,G,C,NaN,NaN,TCGA-D8-A1XY-01A-11D-A14K-09,TCGA-D8-A1XY-10A-01D-A14K-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Somatic,NaN,NaN,NaN,NaN,NaN,Illumina HiSeq 2000,edb6d161-8f50-4c11-8246-487c4ea9a55d,8dea96d9-5017-4872-a84e-33bfd2f37b7a,c.9888C>G,p.=,p.L3296L,ENST00000398571,79/80,149,114,35,152,NaN,NaN,"USP34,synonymous_variant,p.L3296L,ENST00000398...",C,ENSG00000115464,ENST00000398571,Transcrip

In [ ]:
path_id = []
num_mut = []
pathway_name = []

for name, group in kegg.groupby('pathway_id'):
  path_id.append(name)
  num_mut.append(sum(mut_hsa.Entrez_Gene_Id.isin(group.gene_id)))
  pathway_name.append(group.pathway_name)
  # print(name)

In [ ]:
# # this cell takes too long, so use the one above

# # for n, g in kegg.groupby('pathway_id'):
# #     print(n, g)

# path_id = []
# num_mut = []

# for name, group in kegg.groupby('pathway_id'):
#   path_id.append(name)
#   num_mut.append(sum(x in list(group.gene_id) for x in mut_hsa.Entrez_Gene_Id))
# #     print(name, list(group.gene_id))
# #     print(name, sum(x in list(group.gene_id) for x in mut_hsa.Entrez_Gene_Id))

In [ ]:
path_count = np.c_[path_id, num_mut, pathway_name]
path_count = pd.DataFrame(path_count, columns = ['pathway_id', 'num_samples', 'pathway_name'])
print(path_count.shape)  
path_count.head()

(336, 3)


,pathway_id,num_samples,pathway_name
0,path:hsa00010,241,0 Glycolysis / Gluconeogenesis - Homo sapi...
1,path:hsa00020,131,68 Citrate cycle (TCA cycle) - Homo sapiens...
2,path:hsa00030,111,98 Pentose phosphate pathway - Homo sapien...
3,path:hsa00040,125,128 Pentose and glucuronate interconversion...
4,path:hsa00051,133,162 Fructose and mannose metabolism - Homo ...


The cell below gives the number of breast cancer samples that have non-MODIFIER mutations in each KEGG pathway. They are ranked.

In [ ]:
path_count.num_samples = path_count.num_samples.astype(int)
path_count.sort_values('num_samples', ascending=False)

,pathway_id,num_samples,pathway_name
87,path:hsa01100,6593,2966 Metabolic pathways - Homo sapiens (hum...
297,path:hsa05200,4055,27318 Pathways in cancer - Homo sapiens (hu...
294,path:hsa05168,3239,26414 Herpes simplex virus 1 infection - Ho...
291,path:hsa05165,3159,25679 Human papillomavirus infection - Homo...
147,path:hsa04151,3137,10932 PI3K-Akt signaling pathway - Homo sap...
115,path:hsa04010,2246,6241 MAPK signaling pathway - Homo sapiens ...
169,path:hsa04510,2237,13499 Focal adhesion - Homo sapiens (human)...
301,path:hsa05205,2164,28318 Proteoglycans in cancer - Homo sapien...
292,path:hsa05166,2101,26009 Human T-cell leukemia virus 1 infecti...
121,path:hsa04024,1949,7423 cAMP signaling pathway - Homo sapiens ...


The following cells are trying to calculate: Number of patients with mutated pathway / Total number of patients.

In [ ]:
# Get the gene IDs under the top pathway from kegg
kegg_top = kegg[kegg.pathway_id == 'path:hsa01100']
kegg_top.head()

,gene_id,pathway_id,pathway_name
2966,hsa:10,path:hsa01100,Metabolic pathways - Homo sapiens (human)
2967,hsa:100,path:hsa01100,Metabolic pathways - Homo sapiens (human)
2968,hsa:10005,path:hsa01100,Metabolic pathways - Homo sapiens (human)
2969,hsa:10007,path:hsa01100,Metabolic pathways - Homo sapiens (human)
2970,hsa:100137049,path:hsa01100,Metabolic pathways - Homo sapiens (human)


In [ ]:
# Find the gene IDs from cancer samples that match the gene IDs under the top pathway
mut_top = mut_hsa[mut_hsa.Entrez_Gene_Id.isin(kegg_top.gene_id)]
mut_top.head()

,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,dbSNP_RS,dbSNP_Val_Status,Tumor_Sample_Barcode,Matched_Norm_Sample_Barcode,Match_Norm_Seq_Allele1,Match_Norm_Seq_Allele2,Tumor_Validation_Allele1,Tumor_Validation_Allele2,Match_Norm_Validation_Allele1,Match_Norm_Validation_Allele2,Verification_Status,Validation_Status,Mutation_Status,Sequencing_Phase,Sequence_Source,Validation_Method,Score,BAM_File,Sequencer,Tumor_Sample_UUID,Matched_Norm_Sample_UUID,HGVSc,HGVSp,HGVSp_Short,Transcript_ID,Exon_Number,t_depth,t_ref_count,t_alt_count,n_depth,n_ref_count,n_alt_count,all_effects,Allele,Gene,Feature,Feature_type,One_Consequence,Consequence,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,ALLELE_NUM,DISTANCE,TRANSCRIPT_STRAND,SYMBOL,SYMBOL_SOURCE,HGNC_ID,BIOTYPE,CANONICAL,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,RefSeq,SIFT,PolyPhen,EXON,INTRON,DOMAINS,GMAF,AFR_MAF,AMR_MAF,ASN_MAF,EAS_MAF,EUR_MAF,SAS_MAF,AA_MAF,EA_MAF,CLIN_SIG,SOMATIC,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,IMPACT,PICK,VARIANT_CLASS,TSL,HGVS_OFFSET,PHENO,MINIMISED,ExAC_AF,ExAC_AF_Adj,ExAC_AF_AFR,ExAC_AF_AMR,ExAC_AF_EAS,ExAC_AF_FIN,ExAC_AF_NFE,ExAC_AF_OTH,ExAC_AF_SAS,GENE_PHENO,FILTER,CONTEXT,src_vcf_id,tumor_bam_uuid,normal_bam_uuid,case_id,GDC_FILTER,COSMIC,MC3_Overlap,GDC_Validation_Status
11,PIK3CA,hsa:5290,WUGSC,GRCh38,chr3,179218303,179218303,+,Missense_Mutation,SNP,G,G,A,rs104886003,byCluster,TCGA-D8-A1XY-01A-11D-A14K-09,TCGA-D8-A1XY-10A-01D-A14K-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Somatic,NaN,NaN,NaN,NaN,NaN,Illumina HiSeq 2000,edb6d161-8f50-4c11-8246-487c4ea9a55d,8dea96d9-5017-4872-a84e-33bfd2f37b7a,c.1633G>A,p.Glu545Lys,p.E545K,ENST00000263967,10/21,114,70,44,83,NaN,NaN,"PIK3CA,missense_variant,p.E545K,ENST0000026396...",A,ENSG00000121879,ENST00000263967,Transcript,missense_variant,missense_variant,1790/9093,1633/3207,545/1068,E/K,Gag/Aag,rs104886003,1,NaN,1.0,PIK3CA,HGNC,HGNC:8975,protein_coding,YES,CCDS43171.1,ENSP00000263967,P42336,NaN,UPI000013D494,NM_006218.2,deleterious(0.02),probably_damaging(0.959),10/21,NaN,Pfam_domain:PF00613;SMART_domains:SM00145;Supe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,not_provided;pathogenic,NaN,21264207,NaN,NaN,NaN,NaN,MODERATE,1.0,SNV,2.0,NaN,1,1,0.000008,0.000008,0.0,0.0,0.000000,0.0,0.000015,0.0,0.000000,NaN,PASS,TCACTGAGCAG,d083d669-6646-463b-853e-c58da8d06439,4374e19d-c5e7-49cf-8707-05ae5aeb7369,aadee87c-6a68-4580-bd10-64ac273b1e3d,0130d616-885e-4a6c-9d03-2f17dd692a05,NaN,COSM125370;COSM763,True,Unknown
12,MCCC1,hsa:56922,WUGSC,GRCh38,chr3,183092485,183092485,+,Missense_Mutation,SNP,C,C,T,rs569042803,by1000G,TCGA-D8-A1XY-01A-11D-A14K-09,TCGA-D8-A1XY-10A-01D-A14K-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Somatic,NaN,NaN,NaN,NaN,NaN,Illumina HiSeq 2000,edb6d161-8f50-4c11-8246-487c4ea9a55d,8dea96d9-5017-4872-a84e-33bfd2f37b7a,c.197G>A,p.Arg66His,p.R66H,ENST00000265594,3/19,191,165,26,124,NaN,NaN,"MCCC1,missense_variant,p.R66H,ENST00000265594,...",T,ENSG00000078070,ENST00000265594,Transcript,missense_variant,missense_variant,344/2545,197/2178,66/725,R/H,cGc/cAc,rs569042803,1,NaN,-1.0,MCCC1,HGNC,HGNC:6936,protein_coding,YES,CCDS3241.1,ENSP00000265594,Q96RQ3,NaN,UPI000013D646,NM_020166.3,deleterious(0.02),benign(0.276),3/19,NaN,Pfam_domain:PF00289;PROSITE_profiles:PS50979;S...,NaN,0.0,0.0,NaN,0.0,0.0,0.001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MODERATE,1.0,SNV,1.0,NaN,NaN,1,0.000008,0.000008,0.0,0.0,0.000000,0.0,0.000015,0.0,0.000000,NaN,PASS,CTGTGCGCATC,d083d669-6646-463b-853e-c58da8d06439,4374e19d-c5e7-49cf-8707-05ae5aeb7369,aadee87c-6a68-4580-bd10-64ac273b1e3d,0130d616-885e-4a6c-9d03-2f17dd692a05,NaN,COSM1041745,True,Unknown
14,UGT8,hsa:7368,WUGSC,GRCh38,chr4,114665726,114665727,+,Frame_Shift_Del,DEL,AT,AT,-,NaN,NaN,TCGA-D8-A1XY-01A-11D-A14K-09,TCGA-D8-A1XY-10A-01D-A14K-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Somatic,NaN,NaN,NaN,NaN,NaN,Illumina HiSeq 2000,edb6d16

In [ ]:
num_top = mut_top.Tumor_Sample_Barcode.nunique()
num_top
# Number of patients with mutated pathway

924

In [ ]:
num_total = maf.Tumor_Sample_Barcode.nunique()
num_total
# Total number of patients

986

In [ ]:
mut_freq = num_top/num_total
mut_freq
# the mutation frequency of the pathway (Number of patients with mutated pathway / Total number of patients)

0.9371196754563894

The most mutated pathway in the TCGA breast cancer data set is hsa01100 (2966 Metabolic pathways - Homo sapiens). The mutation frequency of this pathway is 0.937 = 93.7%.